### imports

In [1]:
# functional libraries
import numpy as np
import pandas as pd
import scipy
import math
import util
import random
import cv2
import os
import time, datetime
import matplotlib.pyplot as plt
from PIL import Image

# model function
from processing import extract_parts, draw
from model.cmu_model import get_testing_model
from config_reader import config_reader


Using TensorFlow backend.


### model and movement dictionary

In [2]:
# load model
params, model_params = config_reader()
model = get_testing_model()
model.load_weights('model/model.h5')

# dictionary of possible weightlifting movements and corresponding numerical value
movements = {
    'snatch_deadlift': 0,
    'snatch': 1,
    'overhead_squat': 2,
    'power_snatch': 3,
    'deadlift': 4,
    'clean': 5,
    'front_squat': 6,
    'power_clean': 7
}


### getters

In [3]:
# returns the peak data (a collection of the 'peaks' or the position of each joint in the frame) of a frame
#
# |       subject 1 in frame       | subject 2 in frame | ... |
# --------------------------------------------------------------
# | (x, y, weight, n/total joints) |         ...        | ... |
# |             ...                |         ...        | ... |
# |                                |                    |     |
def get_peaks(video, frame):
    return pd.read_pickle('../data/%s_data/%s_peaks' % (video, frame))

# returns the subset data (a subset of the provided peaks where the related joints are grouped together)  of a frame
#
# |       'nose'       | 'neck' | ... |
# --------------------------------------
# | (n/total joints) |    ...   | ... |
# |       ...        |    ...   | ... |
# |                  |          |     |
def get_sub(video, frame):
    return pd.read_pickle('../data/%s_data/%s_subset' % (video, frame))

# returns the candidate data (a list of all the coordinates in a frame, in order) of a frame
#
# |  x  |  y  | weight | (n/total joints) |
# ------------------------------------------
# | ... | ... |  ...   |       ...        |
# |     |     |        |                  |
def get_can(video, frame):
    return pd.read_pickle('../data/%s_data/%s_candidate' % (video, frame))

# returns a list of the frames (formatted as strings - 'frame #') in a given directory
def get_frames(path, delim):
    frames = os.listdir(path) # add all the frames to a list
    frames = list(filter(lambda x: x.split(delim)[0].isdigit(), frames)) # remove none frame files
    frames.sort(key = lambda x: int(x.split(delim)[0])) # sort frames in order
    return frames


In [4]:
# get_peaks('clean_1', 4)
# get_sub('clean_5', 12)
# get_can('clean_8', 61)

### 1. convert a video to frames

In [5]:
# converts a video at 'path' to frame by frame images saved to be edited
def video_to_frames(path, div):
    # name = path.split('/')[len(path.split('/')) - 1].split('.')[0]
    vid = cv2.VideoCapture(path) # load video into a variable
    count = 0 # set the frame count
    
    while(count < vid.get(cv2.CAP_PROP_FRAME_COUNT)):
        ret, frame = vid.read() # frame-by-frame
        
        if count % div != 0:
            count += 1
            continue
        
        if np.all(frame != None):
            # print('saving.. %s to %s' % (file, loc))
            frame = cv2.resize(frame, (910, 1184)) #, fx=0.1, fy=0.1)
            cv2.imwrite('../images/edit/%d.jpg' % (count / div), frame) # save image from the current frame 
        # else:
            # print('invalid file.. ' ) 
        
        count += 1 # counter

    vid.release() # release capture
    cv2.destroyAllWindows() # close the video
    

In [6]:
# video = 'snatch_20'

In [7]:
# video_to_frames('../videos/unprocessed/%s.mov' % video, 2)

### 2. rename the files by movement

In [8]:
# manually rename images in training to their specific movements
def new_name(path, loc, mvmt, start, end):
    count = start
    frames = get_frames(path, '.')
    
    if mvmt == 0:
        while count < len(frames):
            os.rename('%s%s' % (path, frames[count]), 
                      '%s%s.jpg' % (loc, count)) # saves file as frame_movement.jpg
            count += 1
    else:
        while count < end:
            os.rename('%s%s' % (path, frames[count - start]), 
                      '%s%s_%d.jpg' % (loc, count, movements[mvmt])) # saves file as frame_movement.jpg
            count += 1
            

In [9]:
# new_name('../images/edit/', '../images/training/%s/' % video, 'snatch_deadlift', 0, 11)
# new_name('../images/edit/', '../images/training/%s/' % video, 'snatch', 11, 22)
# new_name('../images/edit/', '../images/training/%s/' % video, 'overhead_squat', 22, 59)
# new_name('../images/edit/', '../images/training/%s/' % video, 'power_snatch', 59, 73)
# new_name('../images/edit/', '../images/testing/clean_16/', 0, 0, 100)

### 3. extract joint data

In [10]:
# reads the images in 'path' and extracts the joint peak, subset, and candidate data then saves to the subsequent (already made) file
def frames_to_joints(path, delim):
    name = path.split('/')[len(path.split('/')) - 2] # extract name
    frames = get_frames(path, delim)
    
    os.mkdir('./output/%s/' % name) # make image output location
    os.mkdir('../data/%s_data/' % name) # make data output location 
    
    for i in frames:
        img = cv2.imread(path + i) # read image
        bgr_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # convert image to B, G, R
    
        peaks, subset, candidate = extract_parts(bgr_img, params, model, model_params) # extract peaks, subsets, and candidates
        joint_img = draw(bgr_img, peaks, subset, candidate)
        
        cv2.imwrite('./output/%s/' % i, joint_img) # save image
        pd.DataFrame(peaks).to_pickle('../data/%s_data/%s_peaks' % (name, i.split('_')[0])) # save peaks
        pd.DataFrame(subset).to_pickle('../data/%s_data/%s_subset' % (name, i.split('_')[0])) # save subset
        pd.DataFrame(candidate).to_pickle('../data/%s_data/%s_candidate' % (name, i.split('_')[0]))  # save candidate
    

In [11]:
# frames_to_joints('../images/training/snatch_15/', '_') - training
# frames_to_joints('../images/testing/clean_16/', '.')

### 4. extract the (x, y) of each joint in a frame and append the movement 

In [12]:
# returns a dataframe with the features as the columns with a target at the end and the (x, y) being the feature data and the movement being the target data
def zero_frame_joint_data(name, frame, movement, count):
    can = 0 # defaults to 0 since training data should only have one
    subset = get_sub(name, frame) # get subset data
    candidate = get_can(name, frame) # get candidate data
    
    if can < len(subset): # len(subset) is equivalent to the number of figures mapped in the frame
        xy = []
        subset = subset.loc[can, :(len(subset.loc[0, :]) - 3)].astype(int) # array of all the coordinates pertaining to the candidate 
            
        for j in subset:
            try:
                x = int(candidate.loc[j:j, :0][0]) / 256
                y = int(candidate.loc[j:j, 1:1][1]) / 256
            except:
                try:
                    x = df.loc[count - 1:count - 1, count:count].values[0][0][0] / 256 
                    y = df.loc[count - 1:count - 1, count:count].values[0][0][1] / 256 
                except: 
                    x = None
                    y = None
            xy.append((x, y)) # append (x, y) of each joint
            
        # grip and elbow width
        try:
            xy.append(math.sqrt((xy[4][0] - xy[7][0])**2 + (xy[7][1] - xy[7][1])**2)) # Rwri_Lwri
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[3][0] - xy[6][0])**2 + (xy[3][1] - xy[6][1])**2)) # Relb_Lelb
        except:
            xy.append(None)
            
        # arm measurements 
        try:
            xy.append(math.sqrt((xy[4][0] - xy[3][0])**2 + (xy[4][1] - xy[3][1])**2)) # Rwri_Relb
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[4][0] - xy[2][0])**2 + (xy[4][1] - xy[2][1])**2)) # Rwri_Rsho
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[3][0] - xy[2][0])**2 + (xy[3][1] - xy[2][1])**2)) # Relb_Rsho
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[7][0] - xy[6][0])**2 + (xy[7][1] - xy[6][1])**2)) # Lwri_Lelb
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[7][0] - xy[5][0])**2 + (xy[7][1] - xy[5][1])**2)) # Lwri_Lsho
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[6][0] - xy[5][0])**2 + (xy[6][1] - xy[5][1])**2)) # Lelb_Lsho
        except:
            xy.append(None)
            
        # leg measurements
        try:
            xy.append(math.sqrt((xy[10][0] - xy[9][0])**2 + (xy[10][1] - xy[9][1])**2)) # Rank_Rkne
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[10][0] - xy[8][0])**2 + (xy[10][1] - xy[8][1])**2)) # Rank_Rhip
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[9][0] - xy[8][0])**2 + (xy[9][1] - xy[8][1])**2)) # Rkne_Rhip
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[13][0] - xy[12][0])**2 + (xy[13][1] - xy[12][1])**2)) # Lank_Lkne
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[13][0] - xy[11][0])**2 + (xy[13][1] - xy[11][1])**2)) # Lank_Lhip
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[12][0] - xy[11][0])**2 + (xy[12][1] - xy[11][1])**2)) # Lkne_Lhip
        except:
            xy.append(None)
            
        # wrist to leg measurements     
        try:
            xy.append(math.sqrt((xy[4][0] - xy[8][0])**2 + (xy[4][1] - xy[8][1])**2)) # Rwri_Rhip
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[4][0] - xy[9][0])**2 + (xy[4][1] - xy[9][1])**2)) # Rwri_Rkne
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[4][0] - xy[10][0])**2 + (xy[4][1] - xy[10][1])**2)) # Rwri_Rank
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[7][0] - xy[11][0])**2 + (xy[7][1] - xy[11][1])**2)) # Lwri_Lhip
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[7][0] - xy[12][0])**2 + (xy[7][1] - xy[12][1])**2)) # Lwri_Lkne
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[7][0] - xy[13][0])**2 + (xy[7][1] - xy[13][1])**2)) # Lwri_Lank
        except:
            xy.append(None)
            
            
        # elbow to leg measurements    
        try:
            xy.append(math.sqrt((xy[3][0] - xy[9][0])**2 + (xy[3][1] - xy[9][1])**2)) # Relb_Rkne
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[3][0] - xy[10][0])**2 + (xy[3][1] - xy[10][1])**2)) # Relb_Rank
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[6][0] - xy[12][0])**2 + (xy[6][1] - xy[12][1])**2)) # Lelb_Lkne
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[6][0] - xy[13][0])**2 + (xy[6][1] - xy[13][1])**2)) # Lelb_Lank
        except:
            xy.append(None)

        # height measurements
        try:
            xy.append(math.sqrt((xy[0][0] - xy[10][0])**2 + (xy[0][1] - xy[10][1])**2)) # nose_Rank
        except:
            xy.append(None) 
        try:
            xy.append(math.sqrt((xy[0][0] - xy[13][0])**2 + (xy[0][1] - xy[13][1])**2)) # nose_Lank
        except:
            xy.append(None) 
        try:
            xy.append(math.sqrt((xy[0][0] - xy[9][0])**2 + (xy[0][1] - xy[9][1])**2)) # nose_Rkne
        except:
            xy.append(None) 
        try:
            xy.append(math.sqrt((xy[0][0] - xy[12][0])**2 + (xy[0][1] - xy[12][1])**2)) # nose_Lkne
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[0][0] - xy[8][0])**2 + (xy[0][1] - xy[8][1])**2)) # nose_Rhip
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[0][0] - xy[11][0])**2 + (xy[0][1] - xy[11][1])**2)) # nose_Lhip
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[0][0] - xy[4][0])**2 + (xy[0][1] - xy[4][1])**2)) # nose_Rwri
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[0][0] - xy[7][0])**2 + (xy[0][1] - xy[7][1])**2)) # nose_Lwri
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[0][0] - xy[3][0])**2 + (xy[0][1] - xy[3][1])**2)) # nose_Relb
        except:
            xy.append(None)
        try:
            xy.append(math.sqrt((xy[0][0] - xy[6][0])**2 + (xy[0][1] - xy[6][1])**2)) # nose_Lelb
        except:
            xy.append(None)
            
        xy.append(movement) # append the movement 
        
        return pd.DataFrame(np.array(xy)).T # convert to passable dataframe
    else:
        return None
        

In [13]:
### TRAINING ###

In [14]:
# video = 'pwrsnatch_8'

# frames = get_frames('../images/training/%s/' % video, '_')
# df = pd.DataFrame() # video joint data
# count = 0
# div = 1

# for i in frames:
    # if count % div != 0:
        # count += 1
        # continue
    
    # frame = i.split('_')[0]
    # movement = i.split('_')[1].split('.')[0]

    # df = df.append(zero_frame_joint_data('%s' % video, frame, movement, count / div), ignore_index = True)
    # count += 1
    
# df.to_csv('../data/2d/%s_2d_data.csv' % video)


In [15]:
### TESTING ###

In [16]:
# video = 'test_2'

# frames = get_frames('../images/testing/%s/' % video, '.')
# df = pd.DataFrame() # video joint data
# count = 0
# div = 1

# for i in frames:
    # if count % div != 0:
        # count += 1
        # continue
        
    # df = df.append(zero_frame_joint_data('%s' % video, i, 100, count / div), ignore_index = True)
    # count += 1

# df.to_csv('../data/2d/%s_2d_data.csv' % video)
    

### 5. convert the 2D data to data over time

In [17]:
def time_data(data):
    new = []
    count = 0
    
    while count < len(data) - 1:
        temp = []
        cur = data.values.tolist()[count] # values for the current frame
        nxt = data.values.tolist()[count + 1] # values for the proceeding frame 
        
        for i in range(len(cur)):
            if i < 18: # first 18 are joint coordinates 
                try:
                    temp.append((nxt[i][1] - cur[i][1]) / (nxt[i][0] - cur[i][0])) # append slope
                except:
                    temp.append(0) # slope may be 0
            elif i < len(cur) - 1:
                try:
                    temp.append((nxt[i] + cur[i]) / 2) # appends the avg distance between the two frames
                    temp.append(nxt[i] - cur[i]) # appends the change in distance between the two frames
                except:
                    temp.append(None)
                    temp.append(None)
            else:
                temp.append(cur[i]) # append movement
        
        new.append(temp) # append data and move to next frame
        count += 1
    
    return pd.DataFrame(new)
            

In [18]:
# time_data(df).fillna(0).to_csv('../data/time/%s_time_data.csv' % video)

### 6. process 

In [19]:
def gather_train_data(video):
    tic = time.time()
    # frames_to_joints('../images/training/%s/' % video, '_') # exrtract joint data - uncomment to run through model
    frames = get_frames('../images/training/%s/' % video, '_')
    video_joint_data = pd.DataFrame() # video joint data
    count = 0

    for i in frames:
        frame = i.split('_')[0]
        movement = i.split('_')[1].split('.')[0]
        video_joint_data = video_joint_data.append(zero_frame_joint_data(video, frame, movement, count), ignore_index = True) # add joint coordinate and distance data to the video array
        count += 1
    
    time_data(video_joint_data).to_csv('./output/data/%s_time_data' % video) # calculate and save the features over the course of the video
    toc = time.time()
    print('time: %.2f' % (toc - tic))


In [20]:
# gather_train_data('snatch_1')

In [21]:
def gather_test_data(video, fps_factor):
    tic = time.time()
    #video_to_frames('../videos/unprocessed/%s.mov' % video, fps_factor)
    #new_name('../images/edit/', '../images/testing/%s/' % video, 0, 0, 100)
    #frames_to_joints('../images/testing/%s/' % video, '.') # exrtract joint data - uncomment to run through model
    frames = get_frames('../images/testing/%s/' % video, '.')
    video_joint_data = pd.DataFrame() # video joint data
    count = 0

    for i in frames:
        video_joint_data = video_joint_data.append(zero_frame_joint_data(video, i, 100, count), ignore_index = True) # add joint coordinate and distance data to the video array
        count += 1
    
    time_data(video_joint_data).to_csv('../data/time/%s_time_data.csv' % video) # calculate and save the features over the course of the video
    toc = time.time()
    print('time: %.2f' % (toc - tic))


In [22]:
# gather_test_data('test_2', 4)

In [23]:
# img = cv2.imread('./zzz.jpg') # read image
# foo = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # convert image to B, G, R
# foo = cv2.resize(foo, (128, 128))#, fx=0.1, fy=0.1)
# plt.imshow(foo)

# tic = time.time()
# peaks, subset, candidate = extract_parts(foo, params, model, model_params)
# joint_img = draw(foo, peaks, subset, candidate)  
# toc = time.time()
# print('time: %.2f' % (toc - tic))
    
# plt.imshow(joint_img)
